<a href="https://colab.research.google.com/github/ced-sys/SubTerra/blob/main/align_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
data_path='/content/drive/MyDrive/SubTerra/Data For ML'
files=os.listdir(data_path)

print("Files found in 'Data For ML' folder:")
for f in files:
  print("-", f)

Files found in 'Data For ML' folder:
- slope.tif
- kenya_lithology_clipped.tif
- land_cover_2020.tif
- ndvi_2020.tif
- lst_2020.tif
- heat_flow_kenya.tif
- kenya_dem.tif
- vegetation_peak.tif
- tri.tif
- fault_distance.tif


In [3]:
!pip install rasterio numpy --quiet


In [4]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

In [5]:
input_dir='/content/drive/MyDrive/SubTerra/Data For ML/'
output_dir='/content/drive/MyDrive/SubTerra/aligned'
reference_raster=os.path.join(input_dir, 'kenya_dem.tif')

In [6]:
os.makedirs(output_dir, exist_ok=True)

In [7]:
to_align=[
    'slope.tif',
    'kenya_lithology_clipped.tif',
    'land_cover_2020.tif',
    'ndvi_2020.tif',
    'lst_2020.tif',
    'heat_flow_kenya.tif',
    'vegetation_peak.tif',
    'tri.tif',
    'fault_distance.tif'
]

In [8]:
with rasterio.open(reference_raster) as ref:
  ref_crs=ref.crs
  ref_transform=ref.transform
  ref_width=ref.width
  ref_height=ref.height


In [9]:
for filename in to_align:
  input_path=os.path.join(input_dir, filename)
  output_path=os.path.join(output_dir, f'aligned_{filename}')

  with rasterio.open(input_path) as src:
    dst_array=src.read(
        out_shape=(src.count, ref_height, ref_width),
        resampling=Resampling.bilinear if src.dtypes[0]!= 'uint8' else Resampling.nearest
    )

    kwargs=src.meta.copy()
    kwargs.update({
        'crs':ref_crs,
        'transform':ref_transform,
        'width':ref_width,
        'height':ref_height
    })

    with rasterio.open(output_path, 'w', **kwargs) as dst:dst.write(dst_array)

  print(f"aligned: {filename}-> aligned_{filename}")
print("\nAll rasters aligned successfully and saved in: 'SubTerra/aligned/'")



aligned: slope.tif-> aligned_slope.tif
aligned: kenya_lithology_clipped.tif-> aligned_kenya_lithology_clipped.tif
aligned: land_cover_2020.tif-> aligned_land_cover_2020.tif
aligned: ndvi_2020.tif-> aligned_ndvi_2020.tif
aligned: lst_2020.tif-> aligned_lst_2020.tif
aligned: heat_flow_kenya.tif-> aligned_heat_flow_kenya.tif
aligned: vegetation_peak.tif-> aligned_vegetation_peak.tif
aligned: tri.tif-> aligned_tri.tif
aligned: fault_distance.tif-> aligned_fault_distance.tif

All rasters aligned successfully and saved in: 'SubTerra/aligned/'
